In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats, linalg
import matplotlib.pyplot as plt
from idtxl.bivariate_pid import BivariatePID
from idtxl.data import Data

from mesostat.metric.dim3d.partialcorr import partial_corr

# Append base directory
import os,sys
rootname = "conservative-tripartite-testing"
thispath = os.getcwd()
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

import src.null_models_3D as null3D
import src.null_test as nulltest

%load_ext autoreload
%autoreload 2

## PCorr Funictions

In [ ]:
decompLabels = ['unq_s1', 'unq_s2']

def pcorr(x, y, z):
    return {
        'unq_s1': partial_corr(x, z, np.array([y])),
        'unq_s2': partial_corr(y, z, np.array([x]))
    }

In [ ]:
discrFuncDict = null3D.discr_method_dict()

### Testing binning-dependence

In [ ]:
# valThrDict = None
valThrDict = {'unq_s1': None, 'unq_s2': None}

In [ ]:
nData = 10000

taskDict = {
    'yolo': np.array([0,0,0]),
    'norand': np.array([0,0,0.5]),
    'randx': np.array([0.5,0,0.5]),
    'rand': np.array([0.5,0.5,0.5])
}

for taskName, params in taskDict.items():
    print(taskName)
    rezDict = {}

    # Do continuous tests
    for funcName, func in discrFuncDict.items():
        print('-', funcName)
        
        f_data   = lambda: func(nData, *params)
        f_metric = lambda x, y, z: pcorr(x,y,z)

        rezDF   = nulltest.run_tests(f_data, f_metric, decompLabels, nTest=100)
        rezDFsh = nulltest.run_tests(f_data, f_metric, decompLabels, nTest=100, haveShuffle=True)

        nulltest.plot_test_summary(rezDF, rezDFsh, suptitle=funcName, haveEff=False, valThrDict=valThrDict)
        suffix = '' if valThrDict is None else '_withThr'
        plt.savefig(funcName + '_discr_pcorr_summary_'+taskName+suffix+'.svg')
        plt.show()

### Effect of variance

Continuous

In [ ]:
f_metric_discr = lambda x, y, z: pcorr(x,y,z)

In [ ]:
# Do continuous tests
nData = 10000

alphaStratDict = {
    'PureSrc': lambda alpha: [0,0,alpha],
    'ImpureX': lambda alpha: [alpha,0,alpha],
    'Impure' : lambda alpha: [alpha,alpha,alpha],
}

thrMetricDictDict = {
    'H0_orig' : None,
#     'H0_adj' : {'unq_s1': 0.718, 'unq_s2': 0.718}
#     'H0_adj' : {'unq_s1': 0.5155426964271462, 'unq_s2': 0.5155426964271462}
}


for fName, f_data in discrFuncDict.items():
    for alphaStratName, alphaFunc in alphaStratDict.items():
        # Plot constant thresholds for PureSrc
        avgRand = alphaStratName == 'PureSrc'
        
        f_data_eff = lambda alpha: f_data(nData, *alphaFunc(alpha))
        
        for h0type, thrMetricDict in thrMetricDictDict.items():
            print(fName, alphaStratName, h0type)

            nulltest.run_plot_param_effect(f_data_eff, f_metric_discr, decompLabels, fontsize=12,
                                           nStep=1001, nSkipTest=100, nTest=200, alphaRange=(0, 1),
                                           avgRand=avgRand, thrMetricDict=thrMetricDict, plotAlphaSq=False)

            suffix = 'n_' + str(nData) + '_' + alphaStratName + '_' + h0type

            plt.savefig(fName + '_discr_pcorr_scatter_vareff_'+suffix+'.svg')
            plt.show()

In [ ]:
nData=10000
for fName, f_data in discrFuncDict.items():
    print(fName)
    
    f_data_eff = lambda alpha: f_data(nData, alpha, alpha, alpha)
    nulltest.run_plot_param_effect_test(f_data_eff, f_metric_discr, decompLabels,
                                        nStep=10, nTest=400, alphaRange=(0, 2), valThrDict=valThrDict)
    
    suffix = '' if valThrDict is None else '_withThr'
    plt.savefig(fName + '_discr_pcorr_vareff_n'+str(nData)+suffix+'.png', dpi=200)
    plt.show()

### Effect of data size

In [ ]:
alpha=0.25

# thrLst = [0.6719351345001467,
#  0.6146967156250432,
#  0.5893340729485583,
#  0.5784473458733254,
#  0.5641776108079606,
#  0.5480398857334239,
#  0.536161629340794,
#  0.5292341868435549,
#  0.5214093689544852,
#  0.5155426964271462]

# thrDict = dict(zip((10**np.linspace(2, 4, 10)).astype(int), thrLst))

alphaStratDict = {
    'PureSrc': [0,0,alpha],
    'ImpureX': [alpha,0,alpha],
    'Impure' : [alpha,alpha,alpha],
}

thrMetricDictDict = {
    'H0_orig' : None,
#     'H0_adj' : {'unq_s1': thrDict, 'unq_s2': thrDict}
}


for fName, f_data in discrFuncDict.items():
    for alphaStratName, alphaFunc in alphaStratDict.items():
        f_data_eff = lambda n: f_data(n, *alphaFunc)

        for h0type, thrMetricDict in thrMetricDictDict.items():
            print(fName, alphaStratName, h0type)

            nulltest.run_plot_data_effect(f_data_eff, f_metric_discr, decompLabels,
                                          nStep=101, nSkipTest=10, nTest=200, pVal=0.01,
                                          thrMetricDict=thrMetricDict, fontsize=12)

            suffix = 'alpha_' + str(alpha) + '_' + alphaStratName + '_' + h0type

            plt.savefig(fName + '_discr_pcorr_scatter_nEff_'+suffix+'.svg')
            plt.show()

In [ ]:
alpha=0.5
for fName, f_data in contFuncDict.items():
    print(fName)

    f_data_eff = lambda n: f_data(n=n, aX=alpha, aY=alpha, aZ=alpha)
    nulltest.run_plot_data_effect_test(f_data_eff, f_metric_discr, decompLabels,
                                       nStep=10, nTest=400, valThrDict=valThrDict)
    
    suffix = '' if valThrDict is None else '_withThr'
    plt.savefig(fName + '_discr_pcorr_nEff_sig'+str(sig)+suffix+'.png', dpi=200)
    plt.show()

### Test relationship of unique and redundancy for fixed data size

#### 2. Finding max synergy parameters - GridSearch1D

In [ ]:
from mesostat.visualization.mpl_colors import base_colors_rgb

In [ ]:
tableauColors = base_colors_rgb(key='tableau')

### Redundant Model

In [ ]:
f_data_1D = lambda nData, alpha: null3D.discr_red_noisy(nData, alpha, alpha, alpha)
nDataLst = (10**np.linspace(2, 4, 10)).astype(int)
alphaMaxLst = []
thrAdjLst = []
thrRandLst = []

for nData in nDataLst:
    print(nData)
    alphaMax, thr = nulltest.run_plot_1D_scan(f_data_1D, f_metric_discr, 'unq_s2', 'unq_s1',
                                              varLimits=(0, 1), nData=nData, nStep=100, nTest=100,
                                              colorA = tableauColors[1], colorB = tableauColors[0])
    
    plt.savefig('red_discr_pcorr_1Dscan_unq_n_'+str(nData)+'.svg')
    plt.show()
    
    # Get also shuffle distribution at this alpha
    datagen_func_noparam = lambda nData: f_data_1D(nData, alphaMax)
    randValues = nulltest.sample_decomp(datagen_func_noparam, f_metric_discr, 'unq_s1',
                                        nData=nData, nSample=10000, haveShuffle=True)
    
    alphaMaxLst += [alphaMax]
    thrAdjLst += [thr]
    thrRandLst += [np.quantile(randValues, 0.99)]
    
plt.figure()
# plt.plot(nDataLst, alphaMaxLst, label='param')
plt.plot(nDataLst, thrAdjLst, label='adjusted', color='purple')
plt.plot(nDataLst, thrRandLst, label='shuffle')
plt.legend()
plt.ylim([0, None])
plt.savefig('red_discr_pcorr_1Dscan_unq_summary.svg')
plt.show()

### Synergistic Model

In [ ]:
f_data_1D = lambda nData, alpha: null3D.discr_syn_noisy(nData, alpha, alpha, alpha)
nDataLst = 1000 * np.arange(1, 11)
alphaMaxLst = []
thrAdjLst = []
thrRandLst = []

for nData in nDataLst:
    print(nData)
    alphaMax, thr = nulltest.run_plot_1D_scan(f_data_1D, f_metric_discr, 'unq_s2', 'unq_s1',
                                              varLimits=(0, 1), nData=nData, nStep=100, nTest=100,
                                              colorA = tableauColors[1], colorB = tableauColors[0])
    
    plt.savefig('syn_discr_pcorr_1Dscan_unq_n_'+str(nData)+'.svg')
    plt.show()
    
    
    # Get also shuffle distribution at this alpha
    datagen_func_noparam = lambda nData: f_data_1D(nData, alphaMax)
    randValues = nulltest.sample_decomp(datagen_func_noparam, f_metric_discr, 'unq_s1',
                                        nData=nData, nSample=10000, haveShuffle=True)
    
    alphaMaxLst += [alphaMax]
    thrAdjLst += [thr]
    thrRandLst += [np.quantile(randValues, 0.99)]
    
plt.figure()
# plt.plot(nDataLst, alphaMaxLst, label='param')
plt.plot(nDataLst, thrAdjLst, label='adjusted', color='purple')
plt.plot(nDataLst, thrRandLst, label='shuffle')
plt.legend()
plt.ylim([0, None])
plt.savefig('syn_discr_pcorr_1Dscan_unq_summary.svg')
plt.show()